In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from tabulate import tabulate

class LeNet5_16x16(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5_16x16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2)
        )
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

def load_ensemble(num_models=25):
    ensemble = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for i in range(num_models):
        model = LeNet5_16x16(num_classes=10).to(device)
        model.load_state_dict(torch.load(f"lenet5_trained_model_ensemble_{i+1}.pth", map_location=device))
        model.eval()
        ensemble.append(model)
    
    return ensemble

def load_and_visualize_csv(file_path):
    filename = os.path.basename(file_path)
    subject, digit, run = filename.split('_')[1], filename.split('_')[3], filename.split('_')[5].split('.')[0]
    
    with open(file_path, 'r') as f:
        csv_data = f.read().strip().split('\n')
    
    image_data = np.array([list(map(float, row.split(','))) for row in csv_data])
    return image_data, int(digit)

def preprocess_image(image_data):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    image = Image.fromarray((image_data * 255).astype(np.uint8), mode='L')
    return transform(image).unsqueeze(0)

def ensemble_predict_and_visualize(ensemble, image_tensor):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    image_tensor = image_tensor.to(device)
    
    predictions = []
    for model in ensemble:
        with torch.no_grad():
            output = model(image_tensor)
            pred = torch.argmax(output, dim=1).item()
            predictions.append(pred)
    
    prediction_counts = [predictions.count(i) for i in range(10)]
    
    plt.figure(figsize=(10, 6))
    plt.bar(range(10), prediction_counts)
    plt.title("Ensemble Predictions")
    plt.xlabel("Digit")
    plt.ylabel("Number of Models")
    plt.xticks(range(10))
    
    for i, count in enumerate(prediction_counts):
        plt.text(i, count, str(count), ha='center', va='bottom')
    
    plt.show()
    
    avg_prediction = sum(predictions) / len(predictions)
    sorted_indices = sorted(range(10), key=lambda i: prediction_counts[i], reverse=True)
    top_3_peaks = sorted_indices[:3]
    
    print(f"Average prediction: {avg_prediction:.2f}")
    print(f"Top 3 peaks: {top_3_peaks}")
    
    return predictions, avg_prediction, top_3_peaks

def evaluate_ensemble(csv_folder, ensemble):
    results = []
    total = 0
    correct = 0
    top_1_correct = 0
    top_3_correct = 0

    for file in os.listdir(csv_folder):
        if file.endswith('.txt'):
            total += 1
            file_path = os.path.join(csv_folder, file)
            image_data, true_label = load_and_visualize_csv(file_path)
            image_tensor = preprocess_image(image_data)

            predictions, avg_prediction, top_3_peaks = ensemble_predict_and_visualize(ensemble, image_tensor)
            
            predicted = max(set(predictions), key=predictions.count)
            confidence_scores = [predictions.count(i) / len(predictions) for i in range(10)]
            true_label_confidence = confidence_scores[true_label]
            predicted_confidence = confidence_scores[predicted]
            
            sorted_scores = sorted(enumerate(confidence_scores), key=lambda x: x[1], reverse=True)
            true_label_rank = [i for i, (label, _) in enumerate(sorted_scores) if label == true_label][0] + 1
            predicted_rank = [i for i, (label, _) in enumerate(sorted_scores) if label == predicted][0] + 1
            distance = abs(true_label_rank - predicted_rank)

            if predicted == true_label:
                correct += 1
            if true_label_rank == 1:
                top_1_correct += 1
            if true_label_rank <= 3:
                top_3_correct += 1

            results.append({
                'True Label': true_label,
                'Predicted Label': predicted,
                'Predicted Confidence': f"{predicted_confidence:.4f}",
                'True Label Confidence': f"{true_label_confidence:.4f}",
                'True Label Rank': true_label_rank,
                'Distance': distance
            })

    results_df = pd.DataFrame(results)
    accuracy = (correct / total) * 100
    top_1_accuracy = (top_1_correct / total) * 100
    top_3_accuracy = (top_3_correct / total) * 100

    return results_df, accuracy, top_1_accuracy, top_3_accuracy

In [ ]:
if __name__ == "__main__":
    ensemble = load_ensemble(num_models=25)
    csv_folder = 'CSV_Images'  # Adjust this to your CSV images folder path
    
    evaluation_results, accuracy, top_1_accuracy, top_3_accuracy = evaluate_ensemble(csv_folder, ensemble)
    
    print(tabulate(evaluation_results.head(10), headers='keys', tablefmt='pretty', showindex=False))
    print(f"\nOverall Accuracy: {accuracy:.2f}%")
    print(f"Top-1 Accuracy (true label is model's top prediction): {top_1_accuracy:.2f}%")
    print(f"Top-3 Accuracy (true label is within model's top 3 predictions): {top_3_accuracy:.2f}%")